This notebook queries data from our twitter-topic-sentiment Flask api and displays some results.

In [2]:
import requests
import json

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# from wordcloud import WordCloud

### Request to our FLASK API 

In [4]:
REST_API_URL = "URL_OF_YOUR_FLASK_SERVER" #path to the server exposing your FLASK API
request_url = REST_API_URL + "sentiment"

# REST_API_URL = "http://127.0.0.1:5000/" #path to the server exposing your FLASK API
# request_url = REST_API_URL + "test_params"

In [31]:
hashtag = 'twitter'
request_parameters = {'hashtag': hashtag, 'nb_days': 7, 'nb_tweets': 70,
                      'get_topic_words': "1",'n_topics': 10,'n_words_per_topic': 10,
                      'lda_passes': 2,'return_tweets': "0",'language':'en'}
response = requests.get(request_url,params=request_parameters)
api_results = response.json()

### Sentiment visualization

In [ ]:
tweets_sentiments_df = pd.read_json(api_results["sentiments_json"])
tweets_sentiments_df.created_at = pd.to_datetime(tweets_sentiments_df.created_at).dt.date
tweets_sentiments_df["NEGATIVE"] =  - tweets_sentiments_df["NEGATIVE"]

In [ ]:
plt.figure(figsize=(25,5))
sns.barplot(x="created_at",y="POSITIVE",color = "green",label="positive sentiment",ci=None,data=tweets_sentiments_df)
ax = sns.barplot(x="created_at",y="NEGATIVE",color = "red",label="negative sentiment",ci=None,data=tweets_sentiments_df)

ax.set_xlabel("created_at",fontsize=20,color="gray")
ax.set_ylabel("sentiment",fontsize=20,color="gray")
ax.tick_params(labelsize=15,labelcolor="white")

plt.xticks(rotation=10)
plt.title("Sentiment polarity (over {} tweets) during last week for hastag {} : ".format(len(tweets_sentiments_df),hashtag),color="white",fontsize=20)
plt.legend()
plt.show()

### Topic visualization

In [ ]:
top_topics_df = pd.read_json(api_results["top_topics_json"])

In [ ]:
### Word Cloud
topic_num = 0
topic_to_plot = top_topics_df.loc[top_topics_df.topics==topic_num]
frequencies_dict = {topic_to_plot.iloc[i]["words"]:topic_to_plot.iloc[i]["proba"] for i in range(len(topic_to_plot))}

wordcloud = WordCloud()
wordcloud.generate_from_frequencies(frequencies_dict)

plt.figure(figsize=(20,5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()